# Preparation

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

### Department of Management

In [2]:
r_mgt=requests.get('https://www.lse.ac.uk/management/people-home')
soup_mgt=BeautifulSoup(r_mgt.content,'lxml')

In [3]:
mgt=[]

label1=soup_mgt.find_all('h1')
label1_wanted1=['academic staff',
               'other academic and research staff']
label1_wanted2=['professional services staff']

for label1 in soup_mgt.find_all('h1'):
    department='management'
    
    if label1.get_text().lower() in label1_wanted1:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordionContainer'})
        for label2 in shortcut.find_all('h2',attrs={'class':'accordion__title'}):
            inlabel2=label2.find_next('div',attrs={'class':'accordion__content'})
            for person in inlabel2.find_all('div',attrs={'class':'accordion__txt'}):
                if person.find('p').find('strong'):
                    name=person.find('p').find('strong').get_text()
                    mgt.append([name,department,label]) 
                else:
                    name=person.find('p').find_next('a').get_text()
                    mgt.append([name,department,label])
    
    if label1.get_text().lower() in label1_wanted2:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordionContainer'})
        for label2 in shortcut.find_all('h2',attrs={'class':'accordion__title'}):
            inlabel2=label2.find_next('div',attrs={'class':'accordion__content'})
            for person in inlabel2.find_all('div',attrs={'class':'accordion__txt'}):
                if person.find('p').find('strong'):
                    name=person.find('p').find('strong').get_text()
                    mgt.append([name,department,label]) 
                elif person.find('p').find('b'):
                    name=person.find('p').find('b').get_text()
                    mgt.append([name,department,label]) 
                elif person.find('p').find('span'):
                    name=person.find('p').find('span').get_text()
                    mgt.append([name,department,label]) 
                else:
                    print('not all included')
   



In [4]:
for i in range(len(mgt)):
    namestr=mgt[i][0].lower().split()
    if 'dr' in namestr:
        mgt[i].append('Dr')
    elif 'professor' in namestr:
        mgt[i].append('Professor')
    else:
        mgt[i].append('Non')

    name=mgt[i][0].replace('Dr','').replace('Professor','').replace('Sir','').replace('\xa0',' ').split()
    name=" ".join(name)
    mgt[i][0]=name

In [5]:
mgt=pd.DataFrame(mgt,columns=['name','department','label','title'])
print('\nSome names called vacancies are accidentally included, as they directly appear on the webpage.')
display(mgt[mgt['name']=='Vacancy'])
print('\n\nThere is one name missing because of the abnormal structure of the webpage')
display(mgt[mgt['name']==''])
display(mgt[148:151])

,name,department,label,title
0,Bethania Antunes,management,Academic staff,Dr
1,Sarah Ashwin,management,Academic staff,Professor
2,Jonathan E. Booth,management,Academic staff,Dr
3,Wafaa Elmezraoui,management,Academic staff,Non
4,Karin King,management,Academic staff,Dr
...,...,...,...,...
168,Paul Willman,management,Other academic and research staff,Professor
169,Mohamed Abouaziza,management,Other academic and research staff,Dr
170,Anushri Gupta,management,Other academic and research staff,Dr
171,Philipp Schoenegger,management,Other academic and research staff,Dr


In [7]:
mgt['name'] = mgt['name'].replace('Vacancy', pd.NA)
mgt=mgt.copy().dropna()
mgt.loc[mgt['name']=='','name']='Dr Henry Hang Shen'

display(mgt[mgt['name']=='Vacancy'])
display(mgt[mgt['name']==''])

,name,department,label,title


,name,department,label,title


In [8]:
mgt

,name,department,label,title
0,Bethania Antunes,management,Academic staff,Dr
1,Sarah Ashwin,management,Academic staff,Professor
2,Jonathan E. Booth,management,Academic staff,Dr
3,Wafaa Elmezraoui,management,Academic staff,Non
4,Karin King,management,Academic staff,Dr
...,...,...,...,...
168,Paul Willman,management,Other academic and research staff,Professor
169,Mohamed Abouaziza,management,Other academic and research staff,Dr
170,Anushri Gupta,management,Other academic and research staff,Dr
171,Philipp Schoenegger,management,Other academic and research staff,Dr


### Department of Government

In [11]:
r_gvt=requests.get('https://www.lse.ac.uk/government/people')
soup_gvt=BeautifulSoup(r_gvt.content,'lxml')

In [12]:
gvt=[]

label1=soup_gvt.find_all('h2',attrs={'class':'accordion__title'})
llabel1_wanted1=['academic staff',
               'professional services staff',
                'research staff']
label1_wanted2=['guest teachers and gtas']
label1_wanted3=['emeritus, affiliated & visiting academic staff']

for label1 in soup_gvt.find_all('h2',attrs={'class':'accordion__title'}):
    department='government'
    if label1.get_text().lower() in llabel1_wanted1:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for person in shortcut.find_all('div',attrs={'class':'accordion__txt'}):
            if person.find('p').find('strong'):
                name=person.find('p').find('strong').get_text()
                gvt.append([name,department,label])
            elif person.find('p').find('b'):
                name=person.find('p').find('b').get_text()
                gvt.append([name,department,label])
            else:
                print('not all structure considered')
    
    if label1.get_text().lower() in label1_wanted2:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for person in shortcut.find_all('p'):
            if person.find('strong'):
                continue
            else:
                name=person.get_text()
                gvt.append([name,department,label])

    if label1.get_text().lower() in label1_wanted3:
        label=label1.get_text()
        shortcut=label1.find_next('div',attrs={'class':'accordion__content'})
        for people in shortcut.find_all('ul'):
            for person in people.find_all('li'):
                if person.find('p'):
                    if person.find('p').find('strong'):
                        name=person.find('p').find('strong').get_text()
                        gvt.append([name,department,label])
                    else:
                        name=person.find('p').find('span').get_text()
                        gvt.append([name,department,label])
                else:
                    if person.find('strong'):
                        name=person.find('strong').get_text()
                        gvt.append([name,department,label])
                    else:
                        print('Not all structures considered')
    
    

In [13]:
for i in range(len(gvt)):
    namestr=gvt[i][0].lower().split()
    if '(dr)' in namestr:
        gvt[i].append('Dr')
    elif '(prof)' in namestr:
        gvt[i].append('Professor')
    else:
        gvt[i].append('Non')

    namestrings=[]
    name=gvt[i][0].replace('\xa0',' ').split()
    for stri in name:
        string=stri.strip('()')
        if string.startswith('GV'):
            continue
        elif string in ['Dr','Mr','Prof']:
            continue
        else:
            namestrings.append(stri)
        
    name=" ".join(namestrings)
    gvt[i][0]=name

In [14]:
gvt=pd.DataFrame(gvt,columns=['name','department','label','title'])
gvt

,name,department,label,title
0,Victor Agboga,government,Academic Staff,Non
1,Elise Antoine,government,Academic Staff,Dr
2,Paul Apostolidis,government,Academic Staff,Professor
3,Tom Bailey,government,Academic Staff,Non
4,Daniel Berliner,government,Academic Staff,Dr
...,...,...,...,...
158,Lukas Slothuus,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
159,Zeynep Somer Topcu,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
160,Christine Stedtnitz,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
161,Jill Stuart,government,"Emeritus, Affiliated & Visiting Academic Staff",Dr
